In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
load_dotenv('.env',override=True)


True

In [7]:
openai.api_key = os.environ['OPENAI_API_KEY']


In [10]:
IMAGE_PATH = "sample1.png"

response = openai.Image.create_variation(
    image=open(IMAGE_PATH, mode="rb"),
    n=3,
    #size="256x256",
   # response_format="b64_json",
)

APIError: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID a8180172cd86af2fc615f9423d7a7ec7 in your email.) {
  "error": {
    "code": null,
    "message": "The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID a8180172cd86af2fc615f9423d7a7ec7 in your email.)",
    "param": null,
    "type": "server_error"
  }
}
 500 {'error': {'code': None, 'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID a8180172cd86af2fc615f9423d7a7ec7 in your email.)', 'param': None, 'type': 'server_error'}} {'Date': 'Thu,

In [5]:
from langchain.tools import tool 

import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'


In [6]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)


In [7]:
tools = [get_current_temperature, search_wikipedia]

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")


In [18]:
functions =[format_tool_to_openai_function(f) for f in tools]

model = ChatOpenAI(temperature=0).bind(functions = functions)

In [36]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [44]:
from langchain.agents.format_scratchpad import format_to_openai_functions

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain


In [45]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [46]:
agent_executor.invoke({"input" : "Hi my name is Rahul"})



> Entering new AgentExecutor chain...
Hello Rahul! How can I assist you today?

> Finished chain.


{'input': 'Hi my name is Rahul',
 'chat_history': [HumanMessage(content='Hi my name is Rahul'),
  AIMessage(content='Hello Rahul! How can I assist you today?'),
  HumanMessage(content='Can you tell me capital of India ?'),
  AIMessage(content='The capital of India is New Delhi. It replaced Calcutta as the capital city in 1911.'),
  HumanMessage(content='whats my name'),
  AIMessage(content="I'm sorry, but I don't have access to personal information."),
  HumanMessage(content='whats my name'),
  AIMessage(content="I'm sorry, but I don't have access to personal information."),
  HumanMessage(content='whats the capital of India?'),
  AIMessage(content='The capital of India is New Delhi. It replaced Calcutta as the capital city in 1911.'),
  HumanMessage(content='whats my name'),
  AIMessage(content="I'm sorry, but I don't have access to personal information."),
  HumanMessage(content='Hi my name is Rahul'),
  AIMessage(content='Hello Rahul! How can I assist you today?'),
  HumanMessage(co

In [40]:
#agent_executor.invoke({"input" : "Can you tell me capital of India ?"})

In [41]:
#agent_executor.invoke({"input": "whats the capital of India?"})

In [47]:
agent_executor.invoke({"input": "whats my name ?"})



> Entering new AgentExecutor chain...
Your name is Rahul.

> Finished chain.


{'input': 'whats my name ?',
 'chat_history': [HumanMessage(content='Hi my name is Rahul'),
  AIMessage(content='Hello Rahul! How can I assist you today?'),
  HumanMessage(content='Can you tell me capital of India ?'),
  AIMessage(content='The capital of India is New Delhi. It replaced Calcutta as the capital city in 1911.'),
  HumanMessage(content='whats my name'),
  AIMessage(content="I'm sorry, but I don't have access to personal information."),
  HumanMessage(content='whats my name'),
  AIMessage(content="I'm sorry, but I don't have access to personal information."),
  HumanMessage(content='whats the capital of India?'),
  AIMessage(content='The capital of India is New Delhi. It replaced Calcutta as the capital city in 1911.'),
  HumanMessage(content='whats my name'),
  AIMessage(content="I'm sorry, but I don't have access to personal information."),
  HumanMessage(content='Hi my name is Rahul'),
  AIMessage(content='Hello Rahul! How can I assist you today?'),
  HumanMessage(conten

In [11]:
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline
from diffusers.utils import load_image

pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipe = pipe.to("cuda")
url = "https://huggingface.co/datasets/patrickvonplaten/images/resolve/main/aa_xl/000000009.png"

init_image = load_image(url).convert("RGB")
prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt, image=init_image).images


ModuleNotFoundError: No module named 'diffusers'

In [12]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 5.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
